## **Instalamos pyspark**

In [1]:
! pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 50.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=3d0acd8f16e81d215258111296efd4bb534b4b5be36b2244f424e031cca89c64
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


## **Llamamos a su libreria y la guardamos en una variable para usarla**

In [2]:
from pyspark import SparkContext
sc = SparkContext()

##**Importamos los datos y los procesamos**

In [3]:
def Leer_texto(pNombreArchivo):
  #creamos un arreglo de tuplas 
  Tuplas=[]
  #procesamos nuestro archivo lo volvemos RDD
  RDD = sc.textFile(pNombreArchivo,100)
  #Iniciamos el numero de twitt
  i=1
  for Linea in RDD.collect():
    #eliminamos los twitt que sean vacios
    if len(Linea) != 0: 
      #Creamos los valores para agregar a la tupla con el valor del numero 
      #de twitt y el twitt
      Tup=(i,Linea)
      #Agregamos la tupla a nuestro arreglo tuplas
      Tuplas.append(Tup)
      #incvremwentamos el valor del twitt
      i+=1
  #devolvemos el arreglo de tuplas
  return Tuplas

In [4]:
#Procesamos nuestros datos
data = Leer_texto('twitts.txt')

## **Implementar en un notebook pyspark usando mapreduce el algoritmo TF-IDF**

## **Hallamos el TF**

1.   **linesRDD=sc.parallelize(data)**

Creamos nuestro archivo RDD para realizar los procesos del algoritmos TF

2.   **flatMap(lambda x: [((x[0],i,len(x[1).split()),1) for i in x[1].split()])**

Primero lo que hacemos es convertir cada palabra de cada twitt en tokens.
Para esto le pasamos la funcion flatMap el cual junta todo los datos en una sola lista.

Para ello le pasamos la funcion lambda, con este agregamos una treipla la cual contiene una tupla en la posicion 0, un valor entero en la posicion 1 y el tamaño del texto en su posicion 2.
La tripla creada almacena el numero del twitt al cual pertecene junto con el token almacenado y el tamaño de ese texto.

Mientras que el valor entero es 1 el cual representa el numero de veces que se repite inicialmente.

3.   **reduceByKey(lambda x,y:x+y)**

Al observar que existen palabras que se repiten varias veces en un solo twitt pasamos a sumarlos para saber asi el numero exacto de veces que aparece en este mismo.

Para eso usamos reduceByKey el cual nos permite hacer operaciones entre elementos que presentan el mismo Key, esto permitiendonos sumar los valor que se repiten en el mismo twitt y y con el mismo token.

Pasamos la funcion (lambda x,y:x+y) para que sume el valor del value de dicho Key.

4.   **map(lambda x: (x[0][1],(x[0][0],x[1])))**

Con este map reordenamos nuestro RDD para asi presentarlo de una mejor forma.

Primero mostraremos el valor del token.

Segundo tenemos una tupla la cual en su posicion 0 contiene el numero de twitt al cual pertenece, mientras que en su posicion 1 ubicamos el numero de veces que se repite el token dentro del twitt.

5.   **.collect()**

Nos permite mostrar nuestro RDD.

In [14]:
linesRDD=sc.parallelize(data)
#
tfreducidoRDD =linesRDD.flatMap(lambda x: [((x[0],i,len(x[1].split())),1) for i in x[1].split()]).reduceByKey(lambda x,y:x+y)
#tf ordenado que muestra el token, numero de twitt, repeticion 
tfRDD = tfreducidoRDD.map(lambda x: (x[0][1],(x[0][0],x[1]/x[0][2])))
tfRDD.collect()

[('stage', (1, 0.1)),
 ('protest', (1, 0.1)),
 ('store', (1, 0.1)),
 ('nyc', (1, 0.1)),
 ('anger', (1, 0.1)),
 ('misplace', (1, 0.1)),
 ('retweet', (1, 0.1)),
 ('apple', (2, 0.08333333333333333)),
 ('intraday', (2, 0.08333333333333333)),
 ('comment', (2, 0.08333333333333333)),
 ('range', (2, 0.08333333333333333)),
 ('premium', (2, 0.08333333333333333)),
 ('user', (2, 0.08333333333333333)),
 ('trading', (2, 0.08333333333333333)),
 ('stock', (2, 0.08333333333333333)),
 ('mkt', (2, 0.08333333333333333)),
 ('ios', (3, 0.1111111111111111)),
 ('app', (3, 0.2222222222222222)),
 ('brain', (3, 0.1111111111111111)),
 ('iphone', (3, 0.1111111111111111)),
 ('shit', (4, 1.0)),
 ('boss', (5, 0.16666666666666666)),
 ('factcheckthis', (6, 0.5)),
 ('suicide', (8, 0.1111111111111111)),
 ('store', (8, 0.1111111111111111)),
 ('turkey', (8, 0.1111111111111111)),
 ('delete', (9, 0.25)),
 ('music', (9, 0.25)),
 ('customer', (9, 0.25)),
 ('apple', (10, 0.08333333333333333)),
 ('intraday', (10, 0.0833333333333

## **Hallamos el IDF**

1.   **import math**

Importamos la libreria math ya que utilizaremos la funcion log

2.   ** from pyspark.sql.functions import *  **

importamos las functions de pySpark ya que esta nos permitira usar el reduce 

3.   **.map(lambda x: (x[0][1],(x[0][0],x[1],1)))**

Con este map agregaremos un nuveo valor 1 el cual representara la concurrencia del token en todo el corpus

4.   **.map(lambda x:(x[0],x[1][2]))**

Con este map haremos que solo recupere el valor del token y su concurrencia en todo el corpus.

5.   **.reduceByKey(lambda x,y:x+y)**

Con reduceByKey haremos que se sumen los valores de cada token para asi saber su concurrencia en todo el corpus.

6.   **.map(lambda x: (x[0],math.log10(len(data)/x[1])))**

Por ultimo con este map pasaremos la funcon math.log10 para poder hallarlo del tamaño total del corpus entre el numero de veces que aparece un token en este.

In [15]:
import math
from pyspark.sql.functions import *

In [16]:
idfRDD=tfreducidoRDD.map(lambda x: (x[0][1],(x[0][0],x[1],1))).map(lambda x:(x[0],x[1][2])).reduceByKey(lambda x,y:x+y).map(lambda x: (x[0],math.log10(len(data)/x[1])))
idfRDD.collect()

[('stage', 1.3798696809532192),
 ('protest', 1.377146822617746),
 ('store', 1.105595616949241),
 ('nyc', 1.353380100659997),
 ('anger', 1.388142206919209),
 ('misplace', 1.390935107103379),
 ('retweet', 1.399423217328898),
 ('shit', 1.6891722025831901),
 ('boss', 3.104145550554008),
 ('suicide', 3.2802368096096894),
 ('turkey', 3.2802368096096894),
 ('delete', 1.9091689473379532),
 ('music', 1.928054291498327),
 ('customer', 1.968482948553935),
 ('survey', 2.882296800937652),
 ('feedback', 3.104145550554008),
 ('iwatch', 2.627024295834346),
 ('accessory', 2.979206813945708),
 ('gadget', 3.104145550554008),
 ('similar', 2.7361687652594138),
 ('cloud', 2.979206813945708),
 ('onlinefootprint', 3.5812668052736707),
 ('modicumofprudence', 3.5812668052736707),
 ('fuck', 1.535943826487013),
 ('make', 1.928054291498327),
 ('try', 1.9792068139457082),
 ('turn', 2.1662934573028525),
 ('annoying', 2.5812668052736707),
 ('asus', 3.5812668052736707),
 ('freezing', 2.803115554890027),
 ('laggy', 3.5

## **Hallamos TF-IDF**

1.   **.join(idfRDD)**

Lo que hacemos es unir el TF con e IDF por medio de los keys lo cual hara una combinacion total entregando asi los valores de Frecuencia y de el Log10 para asi operar con este

2.   **.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1])))**

Con este map lo que hacemos es ordenar nuestro datos pero ubicando al final la multiplicacion de la frecuencia de un token * el valor de la IDF

3.   **.sortByKey()**

Esto nos permitira realizar la multiplicacion respectiva al numero de twitt que pertenece.

4.   **.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))**

Con este map Mostraremos nuestros datos para su mayo entendimiento.
Primero teniendo el numero de twitt
Segundo el valor del token
Tercero el valor de frecuencia de dicho token
Cuarto el valor del IDF 
Quinto el valor de la frecuencia * idf 


In [17]:
print("Twitt ","Token ", "TF ", "     IDF     ","     TF-IDF     ")
tfidfrdd=tfRDD.join(idfRDD).map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey().map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
tfidfrdd.collect()

Twitt  Token  TF       IDF           TF-IDF     


[(1, 'stage', 0.1, 1.3798696809532192, 0.13798696809532193),
 (1, 'store', 0.1, 1.105595616949241, 0.11055956169492409),
 (1, 'nyc', 0.1, 1.353380100659997, 0.13533801006599971),
 (1, 'protester', 0.1, 1.388142206919209, 0.13881422069192093),
 (1, 'diein', 0.1, 1.399423217328898, 0.1399423217328898),
 (1, 'protest', 0.1, 1.377146822617746, 0.1377146822617746),
 (1, 'anger', 0.1, 1.388142206919209, 0.13881422069192093),
 (1, 'misplace', 0.1, 1.390935107103379, 0.1390935107103379),
 (1, 'retweet', 0.1, 1.399423217328898, 0.1399423217328898),
 (1, 'agree', 0.1, 1.3690792008697128, 0.13690792008697128),
 (2, 'aapl', 0.08333333333333333, 0.5944950710074257, 0.049541255917285476),
 (2,
  'stockaction',
  0.08333333333333333,
  2.0249643045063834,
  0.16874702537553193),
 (2, 'range', 0.08333333333333333, 2.2195389692560776, 0.18496158077133978),
 (2, 'user', 0.08333333333333333, 1.8178388117107334, 0.1514865676425611),
 (2, 'trading', 0.08333333333333333, 1.928054291498327, 0.160671190958193